In [3]:
import pandas as pd
import numpy as np
import re

In [9]:
def preprocess(row):
    row = re.sub('ㅁ', "", row)
    row = re.sub(r"\n", ",", row)
    
    # 용어
    row = re.sub("YoY|yoy|YOY", "전년 대비 증감율", row)
    row = re.sub("QoQ|qoq|QOQ", "전분기 대비 증감율", row)
    row = re.sub("MoM|mom|MOM", "전월 대비 증감율", row)
    row = re.sub("YTD", "연초 누계", row)
    row = re.sub("MTD", "월초 누계", row)
    
        
    # 분기 표현 1Q21F->21년도 1분기 예측값
    quarter_rule2 = re.compile(r'([1-4]Q[\d]{2}F)')
    quarter_text2 = quarter_rule2.finditer(row)
    startIdxList2 = []
    for text in quarter_text2 :
        startIdxList2.append(text.start())
    for i in range(len(startIdxList2)-1, -1, -1) :
        idx = startIdxList2[i]
        quar = row[idx:idx+5]
        new_quar_text = row[idx+2:idx+4]+"년도 "+row[idx]+"분기 예측값"
        row = row.replace(quar, new_quar_text)
    
    # 분기 표현 1Q21->21년도 1분기
    quarter_rule1 = re.compile(r'([1-4]Q[\d]{2})')
    quarter_text1 = quarter_rule1.finditer(row)
    startIdxList1 = []
    for text in quarter_text1 :
        startIdxList1.append(text.start())
    for i in range(len(startIdxList1)-1, -1, -1) :
        idx = startIdxList1[i]
        quar = row[idx:idx+4]
        new_quar_text = row[idx+2:idx+4]+"년도 "+row[idx]+"분기 "
        row = row.replace(quar, new_quar_text)
        
    # 분기 표현 1H21->21년도 상반기
    half_rule = re.compile(r'([12]H[\d]{2})')
    half_text = half_rule.finditer(row)
    startIdxList3 = []
    for text in half_text :
        startIdxList3.append(text.start())
    for i in range(len(startIdxList3)-1, -1, -1) :
        idx = startIdxList3[i]
        half = row[idx:idx+4]
        if(row[idx] == 1) : new_half_text = row[idx+2:idx+4]+"년도 상반기 "
        else : new_half_text = row[idx+2:idx+4]+"년도 하반기 "
        
        row = row.replace(half, new_half_text)
        
    # [내용] 제거
    row = re.sub(r'\[[^)]*\]', '', row)
        
    # 다중 띄어쓰기
    row = re.sub(' +', ' ', row)
    
    # 앞뒤 공백
    row = re.sub(r"^\s+", "", row, flags=re.UNICODE)
    row = re.sub(r"\s+$", "", row, flags=re.UNICODE)
    
    return row

In [11]:
filename = "./combined_dataset_since_20170601.xlsx"
# filename = "./report.xlsx"
df = pd.read_excel(filename)

# print(df.isnull().sum())
# print(df)

logCnt = 1000
plusCnt = 1000
for i in range(len(df)) :
    if(i == logCnt) :
        print(i)
        logCnt += plusCnt
#     print(i, df['title'][i])
    
    # 영어로만 구성된 row는 drop
    if(len(re.findall(u'[가-힣]+', df['title'][i]))>0) :
        df['title'][i] = preprocess(df['title'][i])
    else :
        df = df.drop(i)

df['title'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['title'])
df = df.reset_index(drop=True)
print("Done!")
# print(df[250:260])

C:\Users\user\AppData\Local\Temp/ipykernel_10512/3138663664.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = preprocess(df['title'][i])


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
Done!


In [12]:
df.to_excel("./combined_dataset_cleaned_since_20170601.xlsx", index = False)